In [75]:
names = [
    "/h.0/Add"
]
names = [n + "_output_0" for n in names]
# names += ["/h.0/attn/Split_output_1"]

In [76]:
import onnx

model = onnx.load("models/tinyphysics.onnx")

value_info_protos = []
shape_info = onnx.shape_inference.infer_shapes(model)
for node in shape_info.graph.value_info:
    if node.name in names:
        value_info_protos.append(node)
model.graph.output.extend(value_info_protos)
onnx.checker.check_model(model, full_check=True)
onnx.save(model, "models/tmp_tinyphysics_output.onnx")

In [77]:
onnx.utils.extract_model("models/tmp_tinyphysics_output.onnx", "models/tmp_tinyphysics_extracted.onnx", ["states", "tokens"], names)

In [78]:
# insert node after "/wt2_embedding/Gather" to reshape
node = None
i = -1
for i, n in enumerate(model.graph.node):
    if n.name == "/wt2_embedding/Gather":
        node = n
        break
assert node is not None

reshaped = [1, 2, 0]
model.graph.initializer.extend([
    onnx.helper.make_tensor(
        name="jl_reshape_shape",
        data_type=onnx.TensorProto.INT64,
        dims=[len(reshaped)],
        vals=reshaped,
    )
])
reshape_node = onnx.helper.make_node(
    "Reshape",
    inputs=[node.output[0], "jl_reshape_shape"],
    outputs=["/wt2_embedding/Gather_output_0_reshaped"],
    name="jl_reshape",
)

# reassign input of next node
for j in range(i + 1, len(model.graph.node)):
    for k in range(len(model.graph.node[j].input)):
        if model.graph.node[j].input[k] == node.output[0]:
            model.graph.node[j].input[k] = "/wt2_embedding/Gather_output_0_reshaped"

model.graph.node.insert(i + 1, reshape_node)

onnx.save(model, "models/tmp_tinyphysics_reshaped.onnx")
onnx.checker.check_model(model, full_check=True)

InferenceError: [ShapeInferenceError] Inference error(s): (op_type:Concat, node name: /Concat_1): [ShapeInferenceError] Can't merge shape info. Both inferred and declared dimension have values but they differ. Inferred=2 Declared=20 Dimension=1
(op_type:Add, node name: /Add): [TypeInferenceError] Input 0 expected to have type but instead is null
(op_type:ReduceMean, node name: /h.0/attn/layer_norm/ReduceMean): [TypeInferenceError] Input 0 expected to have type but instead is null
(op_type:Sub, node name: /h.0/attn/layer_norm/Sub): [TypeInferenceError] Input 0 expected to have type but instead is null
(op_type:Pow, node name: /h.0/attn/layer_norm/Pow): [TypeInferenceError] Input 0 expected to have type but instead is null
(op_type:ReduceMean, node name: /h.0/attn/layer_norm/ReduceMean_1): [TypeInferenceError] Input 0 expected to have type but instead is null
(op_type:Add, node name: /h.0/attn/layer_norm/Add): [TypeInferenceError] Input 0 expected to have type but instead is null
(op_typ